In [1]:
%load_ext autoreload
%autoreload 2

In [11]:
import torch
from polysym import Regressor, Operators

n_obs = 1000

X3d = torch.zeros((n_obs, 2, 100))
X2d = torch.zeros((n_obs, 1))
y1d = torch.zeros(n_obs)
y2d = torch.zeros(n_obs, 100)

for obs in range(n_obs):

    start, end = torch.rand(2) * 100

    x1 = torch.linspace(start, end, 100)
    x2 = torch.cos(torch.linspace(start, end, 100))
    b = torch.randint(low=-10, high=10, size=(1, 1))

    y = (torch.cos(x1) + ((x1 * x2/ 10))) + b

    X3d[obs, 0] = x1
    X3d[obs, 1] = x2
    X2d[obs] = b
    y1d[obs] = torch.mean(y).item()
    y2d[obs] = y

operators = Operators(['add', 'sub', 'mul', 'div', 'pow', 'cos',
                       'mean', 'std', 'sin', 'max', 'min'])

regressor2d = Regressor(X3d=X3d,
                        X2d=X2d,
                        y=y2d,
                        operators=operators,
                        max_complexity=5,
                        pop_size=100,
                        max_iter=200,
                        seed=42,
                        verbose=1,
                        workers=-1)


In [12]:
regressor2d.fit()

2025-04-21 18:28:09,238 - INFO - Running iteration 0/200
2025-04-21 18:28:15,798 - INFO - Running iteration 10/200
2025-04-21 18:28:19,892 - INFO - Running iteration 20/200
2025-04-21 18:28:23,700 - INFO - Running iteration 30/200
2025-04-21 18:28:27,661 - INFO - Running iteration 40/200
2025-04-21 18:28:31,141 - INFO - Running iteration 50/200
2025-04-21 18:28:34,203 - INFO - Running iteration 60/200
2025-04-21 18:28:38,213 - INFO - Running iteration 70/200
2025-04-21 18:28:44,406 - INFO - Running iteration 80/200
2025-04-21 18:28:50,526 - INFO - Running iteration 90/200
2025-04-21 18:28:55,929 - INFO - Running iteration 100/200
2025-04-21 18:29:00,885 - INFO - Running iteration 110/200
2025-04-21 18:29:08,022 - INFO - Running iteration 120/200
2025-04-21 18:29:12,762 - INFO - Running iteration 130/200
2025-04-21 18:29:16,840 - INFO - Running iteration 140/200
2025-04-21 18:29:22,552 - INFO - Running iteration 150/200
2025-04-21 18:29:27,650 - INFO - Running iteration 160/200
2025-04-

In [13]:
regressor2d.score()

3.7229678630828857

In [14]:
regressor2d.best_expr

binary_pow(49.02849593388748, binary_mul(v1, v0))

In [15]:
regressor2d.summary()

Best depth=3.8836069107055664 fitness=3.8836069107055664 ; expr=binary_pow(49.02849593388748, binary_mul(v1, v0))
Depth=1 fitness=3.8836069107055664 ; expr=binary_add(v1, x0)
Depth=2 fitness=3.854874849319458 ; expr=binary_add(binary_add(x0, v1), unary_mean(v1))
Depth=3 fitness=3.7229678630828857 ; expr=binary_add(binary_pow(unary_max(x0), unary_cos(v0)), x0)
Depth=4 fitness=6.961439609527588 ; expr=binary_add(v1, unary_min(binary_pow(binary_sub(-51.74724200074575, -54.922626196122444), x0)))
Depth=5 fitness=3.8836069107055664 ; expr=binary_add(binary_sub(unary_mean(x0), unary_mean(unary_max(unary_mean(x0)))), binary_add(x0, v1))
